# 한영 번역 서비스 만들기  M2M100

In [1]:
# !pip install sentencepiece

In [2]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

ko_text = "이것은 한국어/영어 번역기 입니다. m2m100모델로 만들었어요."
en_text = "This is a Korean to English translator. I made it using the m2m100 model."

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# translate Hindi to French
tokenizer.src_lang = "ko"
encoded_ko = tokenizer(ko_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_ko, forced_bos_token_id=tokenizer.get_lang_id("en"))
translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
# => "La vie est comme une boîte de chocolat."
print(translated_text)

# translate Chinese to English
tokenizer.src_lang = "en"
encoded_en = tokenizer(en_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("ko"))
translated_text2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
# => "Life is like a box of chocolate."
print(translated_text2)

/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


This is a Korean/English translator. made with m2m100 model.
이것은 한국어에서 영어로 번역기입니다. m2m100 모델을 사용하여 만들었습니다.


In [1]:
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch_directml  # Import torch-directml for DirectML device
import torch  # Import PyTorch


/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")


In [2]:
dml_device = torch_directml.device()

In [3]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to(dml_device)
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
languages = {
    "Korean": "ko",
    "English": "en",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Japanese": "ja",
    "Chinese (Simplified)": "zh",
    "Russian": "ru",
    # Add more languages as needed
}

def translate(text, source_lang, target_lang):
    tokenizer.src_lang = languages[source_lang]
    encoded_text = tokenizer(text, return_tensors="pt").to(dml_device)  # Send input tensors to DirectML device
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id(languages[target_lang]))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text

with gr.Blocks() as app:
    gr.Markdown("# Multilingual Translation Service")

    with gr.Row():
        source_lang = gr.Dropdown(label="Source Language", choices=list(languages.keys()), value="Korean")
        target_lang = gr.Dropdown(label="Target Language", choices=list(languages.keys()), value="English")

    input_text = gr.Textbox(label="Input Text")
    output_text = gr.Textbox(label="Translated Text")
    translate_btn = gr.Button("Translate")

    translate_btn.click(fn=translate, inputs=[input_text, source_lang, target_lang], outputs=output_text)

app.launch(inline=False, share=True)

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://a20357f81ae2ea2eab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
